# FDNote4W.ipynb

Prepared by Inmoo Lee for the Financial Databases class at KAIST

inmool@kaist.ac.kr

For return calculations using SQL

Input files used

    - 2020SP500Constituents_2025_Short.xlsx
    - note4w.sas7bdat
    - note4Ori.xlsx
    - note4dat.xlsx
    - return_data.ft
Out files
    - fbhrs.ft
    

In [ ]:
import os #import a package called os

print(os.getcwd())  #get the current working directory


# SQL

Let's define a function, pysqldf, to use pandasql's sqldf



In [ ]:
#change the working directory
path='D:\\####'#Change this to your directory to work with
os.chdir(path) # change the working directory
print(os.getcwd())

In [3]:
import numpy as np
import pandas as pd

In [4]:
##Define a function to use pandasql
#pandasql is a package that allows you to use SQL queries on pandas DataFrames
from pandasql import sqldf
def pysqldf(q):
 return sqldf(q, globals())

In [ ]:
# read the data from an Excel file
Table1=pd.read_excel('./note4Ori.xlsx', sheet_name="Table1", header=0)
Table2=pd.read_excel('./note4Ori.xlsx', sheet_name="Table2", header=0)
print(Table1.head())
print(Table2.head())

## Let's join two tables

### left join:

For each row in the left table (i.e., the one in the "from" statement), the columns of the observations that satisfy the conditions specified in the "on" statement will be added

In [ ]:
# Query to join Table1 and Table2 on 'id' and 'GENDER' columns
# The query selects all columns from Table1 and the 'Eye' column from Table2
# Note that the use of 'left join' ensures that all records from Table1 are included
# even if there are no matching records in Table2 

# The records in Table2, which are not in Table1, will not be included in the result
# This is because we are using a left join, which includes all records from the left table

Query='''select a.*,b.Eye 
            from Table1 as a 
            left join Table2 as b 
            on a.id=b.id and a.GENDER=b.GENDER'''

Newtable=pysqldf(Query)


print(Newtable.head())

In [ ]:
###Pandas has a much simpler way to merge two dataframes
## Please check Appendix 1 for more details on pd.merge at the end of this file
# pd.merge is used to merge two DataFrames based on a common column or index

NewtablePD=pd.merge(Table1,Table2,how='left',on=('ID','GENDER'))
print(NewtablePD)


#### Another example

Join two dataframes covering two different time periods

In [ ]:
# read an excel file, Fama-French three-factor values plus S&P 500 index returns: Monthly
Factordat=pd.read_excel('./note4data.xlsx', sheet_name="factors", header=0)
print(Factordat.head())
print(Factordat.tail())
print(Factordat.Date.min())
print(Factordat.Date.max())

In [ ]:
# Exclude observations before 2005
factordat=Factordat[Factordat['Date' ]>= 20050101].copy()
print(factordat.head())
print(factordat.Date.min())
print(factordat.Date.max())

In [ ]:
# read sas data (note42 and exclude thsoe after 2008

df=pd.read_sas('./note4w.sas7bdat')
print(df.head())
print(df.Date.min())
print(df.Date.max())

In [ ]:
stock=df[df['Date']<=20081231].copy()
print(stock.head())
print(stock.Date.min())
print(stock.Date.max())

##### Convert an object columns in bytes to a string before proceeding


In [ ]:
###change bytes to string (For example, the first id is b'IBM'.
### b' indicates that it is in bytes.  To change it to a string value,
### you need to use decode('utf8')))
stock.loc[:,'id']=stock.loc[:,'id'].str.decode('utf8')
stock.loc[:,'isign']=stock.loc[:,'isign'].str.decode('utf8')

print(stock.head())
print(stock.Date.min())
print(stock.Date.max())
print(factordat.Date.min())
print(factordat.Date.max())

### Let's join two tables using "left join".

Here, the columns of the observations satisfying the condition specified in the "on" statement in the "factordat" table are added to the "stock" table

Note that stock covers the data during 2001 - 2008 while factors cover the data during 2005 - 2013, which indicates that the columns from factors will have missing values during 2001-2004 in the final output.

The final output covers the period from 2001 till 2008.

In [ ]:
#Below, we rename the Date column in the factordat DataFrame as factordate to avoid confusion
# with the Date column in the stock DataFrame

L_query='''select a.*,b.Date as factordate,b.SP500,b.RmRf,b.SMB,b.HML,b.RF
           from stock as a
           left join factordat as b
           on a.Date = b.Date
           order by a.Date, a.id'''
Leftjoin=sqldf(L_query,locals())# or Leftjoin=pysqldf(L_query)

print(Leftjoin.head())
print(Leftjoin.tail())
print(stock.Date.min(),factordat.Date.min(),Leftjoin.Date.min()) #Print the minimum date in the joined DataFrame
print(stock.Date.max(),factordat.Date.max(),Leftjoin.Date.max()) #Print the maximum date in the joined DataFrame

In [ ]:
## Check the minimum and maximum dates in the joined DataFrame
print('Join Min Date:\n', Leftjoin[['Date','factordate']].min())
print('Join Max Date:\n  ', Leftjoin[['Date','factordate']].max())

## Check the minimum and maximum dates in the joined DataFrame where SP500 is not null
## Note that you can specify the condition using the .loc method
## Only those rows that satisfy the condition will be included in the result

# Note that the SP500 column is in the factordat DataFram only, so we check the 'factordate' column
print('Join Min Date, Not missing SP500:\n', Leftjoin.loc[Leftjoin.SP500.notnull(),['Date','factordate']].min())
print('Join Max Date, Not missing SP500:\n', Leftjoin.loc[Leftjoin.SP500.notnull(),['Date','factordate']].max())

### Right join

Since right join is not supported, use a trick: Switch tables in "left join"

Different from the previous one, here, the columns of stock's observations satisfying the conditions will be added to "factordat".

Here, the returns will be missing for the period from 2009 in the final data.

The output covers the period from 2005 till 2013

In [ ]:
#Query statement to perofrm a right join
# Since pandasql does not support right join directly, we can use a left join
# and then reverse the order of the tables in the query

# Note that the Date column in the factordat DataFrame is renamed to factordate
# to avoid confusion with the Date column in the stock DataFrame

R_query='''select a.*,b.Date as factordate,b.SP500,b.RmRf,b.SMB,b.HML,b.RF
            from factordat as b
            left join stock as a
            on a.Date = b.Date
            order by b.Date, a.id'''

# All obs in factors are selected in rightjoin*/
Rightjoin=pysqldf(R_query)
print(Rightjoin.head())
print(Rightjoin.tail())

#Check how the data coverage changed after the right join
print(stock.Date.min(),factordat.Date.min(),Rightjoin.factordate.min()) #Print the minimum date in the joined DataFrame
print(stock.Date.max(),factordat.Date.max(),Rightjoin.factordate.max()) #Print the maximum date in the joined DataFrame

In [ ]:
#Check the minimum and maximum dates in the joined DataFrame
print('Join Min Date:\n', Rightjoin[['Date','factordate']].min())
print('Join Max Date:\n', Rightjoin[['Date','factordate']].max())

## Check the minimum and maximum dates in the joined DataFrame where SP500 is not null
## Note that you can specify the condition using the .loc method
## Only those rows that satisfy the condition will be included in the result

## Returns are in the stock DataFrame only, so we check the 'return' column
print('Join Min Date, Not missing returns:\n', Rightjoin.loc[Rightjoin['return'].notnull(),['Date','factordate']].min())
print('Join Max Date, Not missing returns:\n', Rightjoin.loc[Rightjoin['return'].notnull(),['Date','factordate']].max())

## Inner join

Inner join merges two tables's observations that are present in both tables

The output will cover the period during 2005 and 2008

In [ ]:
# only those that are in both stocks and factors are selected*/

I_query='''select a.*,b.Date as factordate,b.SP500,b.RmRf,b.SMB,b.HML,b.RF
            from stock as a
            inner join factordat as b
            on a.Date = b.Date
            order by a.Date, a.id'''
# All obs in factors are selected in rightjoin*/
Innerjoin=sqldf(I_query,locals()) #or pysqldf(I_query)
print(Innerjoin.head())
print(Innerjoin.tail())

In [ ]:
#Check how the data coverage changed after the inner join
print('Join Min Date:\n', Innerjoin[['Date','factordate']].min())
print('Join Max Date:\n', Innerjoin[['Date','factordate']].max())

#Check the minimum and maximum dates in the joined DataFrame where SP500 is not null
print('Join Min Date, Not missing returns and sp500:\n',\
      Innerjoin.loc[Innerjoin['return'].notnull() & Innerjoin.SP500.notnull(),\
                    ['Date','factordate']].min())
print('Join Max Date, Not missing returns and sp500:\n',\
      Innerjoin.loc[Innerjoin['return'].notnull() & Innerjoin.SP500.notnull(),\
                    ['Date','factordate']].max())

## Cartesian join

For each in the "stock" (in from) table, all observations in the "factordat" table (left join) are added (no conditions are specified)

The output covers the period during 2001 and 2013

The total number of observations is # in stock times # in factordat

In [ ]:
# for each observation in stock, all obs in factors are matched*/
# Note that there are no conditions specified in the left join
# This means that all observations in the stock and factodat dataframes will be included


C_query='''select a.*,b.Date as factordate,b.SP500,b.RmRf,b.SMB,b.HML,b.RF
            from stock as a
            left join factordat as b
            order by a.Date, a.id'''
Cartesian =sqldf(C_query,locals()) #or #or pysqldf(C_query)

print(Cartesian.head())
print(Cartesian.tail())#notice that the time period covers only for the one in from

In [ ]:
print('The number of rows:\n', Cartesian.Date.count())
A=stock.Date.count()
B=factordat.Date.count()
print(A,B)
print('# of rows in stock * # of rows in factordat =  ', A*B)

In [ ]:
# Check the minimum and maximum dates in the Cartesian product DataFrame
# Note that the Date column in the factordat DataFrame is renamed to factordate
# to avoid confusion with the Date column in the stock DataFrame
print(Cartesian[['Date','factordate']].min())
print(Cartesian[['Date','factordate']].max())

In [ ]:
# To output certain observations that satisfy some conditions after 
# performing a cartesian product (left join), we can use the following query

# After the Cartesian product, we can select all observations that satisfy the condition
# specified in the WHERE clause``
CW_query='''select a.*,b.Date as factordate,b.SP500,b.RmRf,b.SMB,b.HML,b.RF
            from stock as a
            left join factordat as b
            where a.Date = b.Date
            order by a.Date, a.id'''

Cartesianwhere =sqldf(CW_query,locals()) #or pysqldf(CW_query)
print(Cartesianwhere.head())
print(Cartesianwhere.tail())

print(Cartesianwhere[['Date','factordate']].min())
print(Cartesianwhere[['Date','factordate']].max())
print(stock[['Date']].max())
print(factordat[['Date']].max())
print(stock[['Date']].min())
print(factordat[['Date']].min())

# Calculate buy-and-hold returns

List of input files used here

    - return_data.ft
    - note4data.xlsx

In [ ]:
df=pd.read_feather('./return_data.ft')
print(df.head())
df=df.rename(columns={'return':'ret'})
print(df.head())
print(df.dtypes)

# Calculate holding period returns using SQL

Buy-and-hold returns (BHR) over the n periods can be calculated as follows (e.g., using monthly returns, over 3 years (n=36))

$$
(1+r_{1})\times (1+r_{2})\times ... \times(1+r_{n}) = 1+BHR_{n}
$$

Since SQL does not have multiplications, we have to use a trick to calculate cumulative returns using SQL.

We can convert returns to log returns so that we can use 'sum' instead of 'product'.

Take $log=ln=log_{exp}$ in both sides of the above equation, then 

$$
log((1+r_{1})\times (1+r_{2})\times ... \times(1+r_{n})) = log(1+BHR_{n})
$$

Since $log(x \times y)=log(x)+log(y)$, 

$$
log(1+BHR_n)=log(1+r_1)+log(1+r_2) ... +log(1+r_n)
$$

$log(a)=b$ implies that $exp^b=a$. If we let $log(1+BHR_n)=b$, and  $1+BHR_n = a$, then 

$$
exp^{b}=exp^{log(1+BHR_{n})} = a=1+BHR_{n}
$$  

Therefore, to calculate BHRs, you can sum log returns (=$log(1+r)$) over a range of dates, and then convert it to 1 + BHR by using the exponential function.

In [ ]:
#  calculate the log returns
df.loc[:,'logret']=np.log(1.0+df.loc[:,'ret']) #np.log is for element by element log function
print(df.head())

### Find the month end date

The last transaction date of a month may not be the last day of the month.

We can find the last day of a month using MonthEnd

We can also find the dates a certain number of months before or after the current month using MonthEnd

In [ ]:
## One can find the the month end any # of months before/after a particular date that is a datetime object,
## Using MonthEnd from pandas.tseries.offsets

##To use MonthEnd, we need to import it from pandas.tseries.offsets
from pandas.tseries.offsets import MonthEnd


## If your date column is not a datetime object, you can convert it to a datetime object using pd.to_datetime
### you have to convert the floating 64 format Date to integer and then to string format
### to use pd_to_datetime

#The following code converts the 'Date' column to a datetime object
df.loc[:,'date1']=pd.to_datetime(round(df.loc[:,'Date']).astype(int).astype(str),format='%Y%m%d')+MonthEnd(0)

###find the date 12 months after (+12) and before (-12) the current date
df.loc[:,'fmonth12']=df.loc[:,'date1']+MonthEnd(+12)
df.loc[:,'bmonth12']=df.loc[:,'date1']+MonthEnd(-12)

#Check whether the new columns are created correctly
print(df[['date1','fmonth12','bmonth12']].head())

##### Find an indicator for year and month

In [ ]:
###find yyyymm to be used as an time indicator (year+month)
### since date1 is a datetime variable, dt.year and dt.month
### can be used to get year and month from a date.

df.loc[:,'yyyymm']=df.loc[:,'date1'].dt.year*100+df.date1.dt.month

#Check whether the yyyymm column is created correctly
print(df[['date1','yyyymm']].head())

### The following query will calculate the sum of logret over the period satisfying the conditions specified in "on" statement

##### This is an example of a complicated join, which can be performed in a sql statement

Here, bhrs are calculated for each id and yyyymm specified in "group by" but in addition, they are calculated using all observations in each id and yyyymm category, which satisfy the conditions specified inside the "on" statement.

left join: Notice that we use the same file (df) in both "from" and "left join"



In [ ]:
# As done before, we can add the logrets for a certain number
# of months using SQL.

# Calculate the sum of logrets over the one-year period starting from the current month
# i.e., over the 12 months using SQL
query='''
        select a.id,a.yyyymm,sum(b.logret) as logsum,
            count(b.logret) as nummonths
        from df as a
        left join df as b
        on a.id = b.id and 
            (b.date1 >= a.date1) and 
            (b.date1 < a.fmonth12)
        group by a.id, a.yyyymm
        order by a.id, a.yyyymm
    '''
fbhrs=pysqldf(query)
###convert sum(b.logret) to bhr by taking exp and subtract 1.
fbhrs.loc[:,'bhr1y']=np.exp(fbhrs.logsum)-1.0

# Check the columns and the first 20 rows of the resulting DataFrame
print(fbhrs.columns)
print(fbhrs.head(20))

##### Choose only thoes buy-and-hold returns calculated over 12 months.


In [ ]:
## To treat those with nummonths not being equal to 12 as missing,
## one can use np.where.

## np.where replaces the values of rows of a specified column, 
## which satisfy the condition with the values
## specified (condition, column to replace, value).

#First, check the summary statistics of the bhr1y column
print(fbhrs.bhr1y.describe())

#Exclude those with nummonths not being equal to 12
fbhrs.loc[:,'bhr1y']=np.where(fbhrs.nummonths==12,fbhrs.bhr1y,np.nan)
#Check again the summary statistics of the bhr1y column after excluding those with nummonths not being equal to 12
print(fbhrs.bhr1y.describe())

## Compare the results calculated in the EXCEL, "bhrs" worksheet in note4data.xlsx

As you can see, the first bhr1y calcualted in EXCEL using the function
=PRODUCT(1+C2:C13)-1 is same as the result in bhrs

In [ ]:
## Check whether the bhr1y column is created correctly
## by comparing the results with the ones in the note4data.xlsx file
## that include a 'BHRs' worksheet in which three different ways of calculating
## buy-and-hold returns are shown

print(fbhrs.head())

In [ ]:
# Let's save the file so that we can use it in the next week's note
fbhrs.to_feather('./fbhrs.ft')

# Appendix

- Join dataframes using pd.merge.
- Working with dates in Python
- Read SAS data, convert bytes into string and save the file as a feather file

## Appendix 1: pd.merge

#### You can join dataframes using pd.merge.
 
The following show how "join" can be done using pd.merge

Please check the class note for the comparison between join in padasql and pd.merge


In [ ]:
#Check the columns of the stock and factordat DataFrames
print(stock.columns)
print(factordat.columns)

#To avoid the confusion with the Date column in the stock DataFrame,
# we rename the Date column in the factordat DataFrame as factordate

factordat.rename(columns={'Date': 'factordate'}, inplace=True)

#Check the columns after renaming
print(factordat.columns)

In [ ]:
### Alternatively, use pd.merge
LeftjoinPD=pd.merge(stock,factordat,how='left',left_on=('Date'),right_on=('factordate'))

#sort the DataFrame by Date and id
# This will sort the DataFrame first by 'Date' and then by 'id'
LeftjoinPD.sort_values(by=['Date','id'], ascending=True,inplace=True )
print(LeftjoinPD.head())# pay attention to the first columns (index)

# Reset the index of the DataFrame
# This will reset the index of the DataFrame and drop the old index
LeftjoinPD.reset_index(drop=True,inplace=True)#If you do not use drop=True, old index will be a column

print(LeftjoinPD.head())# pay attention to the first columns (index)
print(LeftjoinPD.tail())# pay attention to the first columns (index)


In [ ]:
#you can use one step, rather than doing it in three steps
#merge, sort, and reset index
# This will merge the stock and factordat DataFrames, sort the resulting DataFrame by '
LeftjoinPD=pd.merge(stock,factordat,how='left',\
                    left_on=('Date'),right_on=('factordate')).sort_values(by=['Date','id'],\
                       ascending=True).\
                    reset_index(drop=True)
##note that I did not use inplace=True since I am assigning the reuslt into a new dataframe
                    
print(LeftjoinPD.head())
print(LeftjoinPD.tail())
##check columns
print(Leftjoin.columns)
print(LeftjoinPD.columns)


In [ ]:
##Right join
## pd.merge has an option to use right join
##Here, there is right join, so do not change the order and use how='right'

print(factordat.columns)
RightjoinPD=pd.merge(stock,factordat,how='right',\
                    left_on=('Date'),right_on='factordate').\
                     sort_values(by=['Date','id'],\
                       ascending=True).\
                    reset_index(drop=True)
##note that I did not use inplace=True in sort_values since you need to use it in the last one only
                    
print(RightjoinPD.head())
print(RightjoinPD.tail())
##Alternatively, use pd.merge
##Here, there is right join, so do not change the order and use how='right'

print(factordat.columns)


In [ ]:
##For inner join, you can use how='inner'

InnerjoinPD=pd.merge(stock,factordat,how='inner',\
                    left_on=('Date'),right_on='factordate').\
                     sort_values(by=['Date','id'],\
                       ascending=True).\
                    reset_index(drop=True)
##note that I did not use inplace=True in sort_values since you need to use it in the last one only
                    
print(InnerjoinPD.head())
print(InnerjoinPD.tail())


In [ ]:
## Cartesian product
## for each observation in stock, all obs in factors are matched*/


##using pd.merge for catesian join is more tricky
##You need a common key
##Since two do not have one, create one and then remove it later
stock['key']=0
factordat['key']=0

CartesianPD=pd.merge(stock,factordat,how='outer').\
                     sort_values(by=['Date','id'],\
                       ascending=True).\
                    reset_index(drop=True).drop(columns='key')
print(CartesianPD.head())
print(CartesianPD.tail())